In [2]:
import cv2 as cv
import numpy as np
import pyttsx3

 
cap = cv.VideoCapture(0)
whT = 320
confThreshold =0.5
nmsThreshold= 0.2
 
#### LOAD MODEL
## Coco Names
classesFile = r"C:\Users\hp\Desktop\Final_OD\classes.names"
classNames = []
with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
print(classNames)
## Model Files
modelConfiguration = r"C:\Usersg\hp\Desktop\Final_OD\yolov4-obj.cfg"
modelWeights = r"C:\Users\hp\Desktop\Final_OD\yolov4-obj_3600.weihts"
net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

 
def findObjects(outputs,img):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det[2]*wT) , int(det[3]*hT)
                x,y = int((det[0]*wT)-w/2) , int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))
    
 
    indices = cv.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)
    
 
    for i in indices:
        i = i[0]
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        # print(x,y,w,h)
        cv.rectangle(img, (x, y), (x+w,y+h), (255, 0 , 255), 2)
        cv.putText(img,f'{classNames[classIds[i]].upper()} {int(confs[i]*100)}%',
                  (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        a=classNames[classIds[i]].upper()
        
        engine = pyttsx3.init()
        engine.say(a)
        engine.runAndWait()


        
    
 
 
while True:
    success, img = cap.read()
 
    blob = cv.dnn.blobFromImage(img, 1 / 255, (whT, whT), [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    layersNames = net.getLayerNames()
    outputNames = [(layersNames[i[0] - 1]) for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(outputNames)
    findObjects(outputs,img)
    
   
    
    
 
    cv.imshow('Image', img)
    cv.waitKey(1)

['Lays', 'parle-g', 'tide', 'toothbrush', 'Banana', 'Potato', 'Cup', 'Shopping cart', 'Exit sign']


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-2b5g8ysb\opencv\modules\dnn\src\darknet\darknet_importer.cpp:207: error: (-212:Parsing error) Failed to parse NetParameter file: C:\Usersg\hp\Desktop\Final_OD\yolov4-obj.cfg in function 'cv::dnn::dnn4_v20200609::readNetFromDarknet'
